# Automatic calcium scoring in chest CT using CNNs

## Introduction

A frequently performed task in various types of images is calcium scoring in computed tomography (CT) exams for cardiovascular disease (CVD) risk prediction. Calcium scoring is routinely performed in non-contrast enhanced ECG-triggered cardiac CT, but increasingly applied in other CT exams visualizing the heart.

This document describes a method based on deep convolutional neural networks for automatic detection and segmentation of arterial calcifications in CT scans. This method is developed for low-dose chest CT and has been published in TMI:

> N. Lessmann, B. van Ginneken, M. Zreik, P.A. de Jong, B.D. de Vos, M.A. Viergever, I. Isgum. Automatic calcium scoring in low-dose chest CT using deep neural networks with dilated convolutions. IEEE Transactions on Medical Imaging 2018, volume 37(2), pp. 615-625, [DOI 10.1109/TMI.2017.2769839](https://dx.doi.org/10.1109/TMI.2017.2769839)

Moreover, an extensive evaluation of the method for calcium scoring in multiple Cardiac CT and Chest CT protocols has been performed. This work is submitted to Radiology: 

> S.G.M. van Velzen, N. Lessmann, B.K. Velthuis, I.E.M. Bank, D.H.J.G. van den Bongard, T. Leiner, P.A. de Jong, W.B. Veldhuis, A. Correa, J.G. Terry, J.J. Carr, M.A. Viergever, H.M. Verkooijen, I. Išgum. Deep learning for automatic calcium scoring in CT: Validation using multiple Cardiac CT and Chest CT protocols. Submitted to Radiology. 


## Python packages

Fundamental components on which this code is build are:

* **Python 3.6**
* **Numpy**
* **Scipy**
* **Theano** (0.9) and **Lasagne** (0.2dev)
* **SimpleITK**

Additionally, **visdom** can used to monitor the training progress.

#### This notebook is a step-by-step guide for training and testing the calcium scoring method. 

In [ ]:
# Add subdirectory to include path
import sys
sys.path.insert(0, './src')
print(sys.version)

# Include custom library
import calciumscoring

# Include other libraries
import os
import numpy as np
import shutil

from glob import glob

# Shortcuts
path = os.path

# Define input and scratch directory locations
if path.exists('/home/user'):
    input_dir = '/home/user/input'
    scratch_dir = '/home/user/scratch'
else:
    input_dir = path.join(os.getcwd(), 'input')
    scratch_dir = path.join(os.getcwd(), 'scratch')
    
    if not path.exists(scratch_dir):
        os.makedirs(scratch_dir)


# Use visdom to display learning curves while the model is training?
# Set to empty string ("") to disable
visdom_server = 'http://your_server:8082'

## Imaging data

In the paper by Lessmann et al., data from the National Lung Screening Trial (NLST) was used for development and evaluation. The NLST was a large lung cancer screening trial in the United States that compared chest X-ray with low-dose chest CT for early detection of lung cancer.

For calcium scoring, a subset was selected from the 6000 available baseline scans, i.e., the scans in the first screening round. We used a diverse subset of in total 1744 scans, of which eventually 1687 scans were useable (this was determined by the human observers who manually annotated calcifications in these scans, see "Manual annotation" section below). For further details regarding the subset selection, please refer to the TMI paper cited above.

In the paper by Lessmann et al., experiments were performed with all scans, but also with only scans reconstructed with the softer (smoother images) or with the sharper (higher contrast, but more noise) reconstruction kernels. The following table gives an overview of the kernels present in this dataset:

| Vendor  | Kernel    | Classification  |
| ------- | --------- | --------------- |
| GE      | STANDARD  | Soft            |
|         | BONE      | Sharp           |
|         | LUNG      | Sharp           |
| Siemens | B30f      | Soft            |
|         | B50f      | Sharp           |
|         | B80f      | Sharp           |
| Philips | C         | Soft            |
|         | D         | Sharp           |
| Toshiba | FC10      | Soft            |
|         | FC51      | Sharp           |

This data is summarized in the file "dataset.csv", which we therefore read and parse into a dictionary. An example of this file can be found in the scr folder.


In the paper by van Velzen et al., several (large) datasets were used for evaluation of the calcium scoring method. These were in total 7240 subjects that received a coronary artery calcium (CAC) scoring CT, diagnostic chect CT, PET attenuation correction CT, radiotherapy treatment planning CT, CAC-screening CT or low-dose chest CT. For further details on the datasets, please refer to the Radiology paper cited above. In this paper the performance of the method, as developed by Lessmann et al (trained with low-dose chest CT from the NLST), was evaluated in the different types of CTs. Moreover, it was investigated wether the method can adapt to different types of CT when representative images are added to the training dataset of NLST scans. 

In [ ]:
metadata = calciumscoring.datasets.read_metadata_file(path.join(input_dir, 'dataset.csv'))
print('Loaded metadata of {}/1687 images'.format(len(metadata)))

### Image resampling

In clinical practice, calcium scoring scans are dedicated cardiac scans with cardiac gating and a standardized axial resolution of 3mm slice thickness and 3mm slice spacing (i.e. non-overlapping slices). Chest CT scans are usually reconstructed to much thinner slices. In our dataset, the slice thickness ranged from 1mm to 3mm and the slice spacing from 0.6mm to 3mm. We therefore resampled the original images to a slice thickness of 3mm and a slice spacing of 1.5mm, which results in overlapping slices (which has been shown to improve the reproducibility of calcium scoring compared to the default 3mm spacing).

Because images become softer with increasing slice thickness due to stronger partial volume effects, we immitated partial voluming by using a weighted average resampling strategy. For each new 3mm slice, the overlap with slices at the original resolution is determined and the values from each original slice are weighted accordingly:

<img src="static_figures/weighted_average_resampling.png" alt="Illustration of the weighted average resampling strategy" style="width: 600px;">

Define a function that uses this strategy to preprocess images:

In [ ]:
# The resampler is defined in the python package "calciumscoring"
# Create an instance set to 3mm thick slices with 1.5mm spacing between slices
resampler = calciumscoring.resampling.WeightedAverageImageResampler(3.0, 1.5)

def preprocess_image(image, spacing, origin, metadata):
    # All images should be in feet-to-head orientation, but originally the images were
    # resampled before correcting the orientation of images that were originally head-to-feet.
    # For compatibility reasons, we therefore flip those images back.
    flip_image = metadata['slice_order'] == 'head-to-feet'
    
    if flip_image:
        image = image[::-1, :, :]  # images are represented in order z-y-x !!!
    image, spacing, origin = resampler.resample(image, spacing, origin, metadata['slice_thickness'])
    if flip_image:
        image = image[::-1, :, :]
    
    # Normalize image values since some vendors use values < -1024 outside the FOV of the scan
    image = np.clip(image, -1000, 3096)
    
    return image, spacing, origin

The above code can be used to load the original DICOM images, preprocess them (i.e. resample to 3mm slice thickness) and save them in the scratch directory.

**Warning:** All of the following steps can take a long time, from at least a few hours to 3-4 days!

In [ ]:
# Check if resampled images are available in the input directory, otherwise
# look for the original DICOM files and preprocess (i.e. resample) them
input_imagedir = path.join(input_dir, 'images')
existing_image_files = glob(path.join(input_imagedir, '*.mha'))

if len(existing_image_files) != len(metadata):
    # Relocate image directory to scratch folder
    input_imagedir = path.join(scratch_dir, 'images')
    for image_file in existing_image_files:
        shutil.copy(image_file, input_imagedir)
    
    # Load dicom files, preprocess the image and store in ITK file format
    dicom_dir = path.join(input_dir, 'dicom')
    n_preprocessed = 0
    
    for dicom_file in glob(path.join(dicom_dir, '*.dcm')):
        series_uid = path.basename(dicom_file)[:-4]
        image, spacing, origin = calciumscoring.io.read_dicom_image(dicom_file)
        image, spacing, origin = preprocess_image(image, spacing, origin, metadata[series_uid])
        calciumscoring.io.write_image(path.join(input_imagedir, series_uid + '.mha'), image, spacing, origin)
        n_preprocessed += 1
    
    print('{} images preprocessed and converted to ITK file format'.format(n_preprocessed))

All images now have a standardized slice thickness and slice increment, but can still have different resolution in the x/y-plane. While manual annotations etc were all done on these 3mm images, the images and calcium masks are internally resampled again to the average resolution of 0.66 x 0.66 x 1.5 mm³. For better performance, we can do this once and store the resampled images (this requires around 100 GB):

In [ ]:
input_maskdir = path.join(input_dir, 'annotations')  # contains the reference segmentations (calcium masks)

standard_resolution = (1.5, 0.66, 0.66)  # z, y, x
standard_imagedir = path.join(scratch_dir, 'images_resampled')
standard_maskdir = path.join(scratch_dir, 'annotations_resampled')

# Make sure that both image and mask dir exist
if not path.exists(standard_imagedir):
    os.makedirs(standard_imagedir)
if not path.exists(standard_maskdir):
    os.makedirs(standard_maskdir)

# Iterate over all images and resample them to the standard resolution
n_resampled = 0

for image_file in glob(path.join(input_imagedir, '*.mha')):
    series_uid = path.basename(image_file)[:-4]
    standard_image_file = path.join(standard_imagedir, series_uid + '.mha')
    standard_mask_file = path.join(standard_maskdir, series_uid + '.mha')
    if path.exists(standard_image_file) and path.exists(standard_mask_file):
        continue

    # Load image, resample to standard resolution, save again
    image, spacing, origin = calciumscoring.io.read_image(image_file)
    mask = calciumscoring.io.read_image(path.join(input_maskdir, series_uid + '.mha'), only_data=True)
    
    image = calciumscoring.resampling.resample_image(image, spacing, standard_resolution)
    mask = calciumscoring.resampling.resample_mask(mask, spacing, standard_resolution)
    
    calciumscoring.io.write_image(standard_image_file, image, standard_resolution, origin)
    calciumscoring.io.write_image(standard_mask_file, mask, standard_resolution, origin)
    
    n_resampled += 1

print('{} images and masks resampled to standard resolution'.format(n_resampled))

### Manual reference standard

Calcifications were manually segmented and labeled in all images. Segmentation was done using a region growing tool, that requires a single click per lesion and then uses the standard calcium scoring threshold 130 HU to segment the entire calcified lesion. In the paper by Lessmann et al the observers manually selected individual voxels, where this procedure did not work (due to low image quality, or lesions that extent into multiple vessels, such as both LAD and LCX), the observers manually selected individual voxels. In the paper by van Velzen et al., region growing was used in all scans. Scans in which this was not possible (i.e. due to exessive noise) were excluded. 

The left main coronary artery (LM) was labeled as LAD since it's often impossible to identify LM in ungated scans. 

In the segmentation masks, voxels are labeled as follows and translated into labels 0-6 for the networks:

| Class        | Label (file) | Label (CNNs) |
| ------------ | ------------ | -------------|
| Background   | 0            | 0            |
| LM and LAD   | 8            | 1            |
| LCX          | 9            | 2            |
| RCA          | 10           | 3            |
| Aorta        | 15           | 4            |
| Aortic valve | 53           | 5            |
| Mitral valve | 58           | 6            |

## Method overview

The method consists of two consecutive CNNs (2.5D, see [A.A.A. Setio et al, TMI 2016](https://dx.doi.org/10.1109/TMI.2016.2536809)). The first network analyzes the entire image and classifies all voxels above the standard calcium threshold (>= 130 HU) as either "Background" or as one of the calcium types (see list above). The second network analyzes all detected candidate calcium voxels again and classifies them as either "no calcium" (0) or "calcium" (1).

Steps when training the method:
* Resample image (done above)
* Train CNN1 with all voxels
* Apply CNN1 to all training and validation images
* Train CNN2 with the voxels >= 130 HU and classified as calcium by CNN1

Because the networks are trained with large numbers of scans, only a few scans are kept in memory at a time. After each epoch (a few minibatches), some scans are removed from the memory and replaced by other scans. This strategy is used for training of both networks.


Steps when applying the method to an unseen image:
* Resample image (3mm thick slices, standard x/y resolution)
* Feed image through CNN1
* Feed parts of the image through CNN2
* Calculate calcium scores
* Determine risk category based on coronary calcium score

## Network 1: Candidate detection and anatomical labeling

<img src="static_figures/CNN1.png" alt="Architecture of the network" style="width: 600px; margin-top: 20px; margin-bottom: 10px;">

Training the first stage network (this will take 3-4 days!):

In [ ]:
%run src/TrainNetwork1.py --inputdir $input_dir --scratchdir $scratch_dir --visdom $visdom_server

Apply the first stage network to the data (will also take long, 1-2 days!):

In [ ]:
%run src/FindCandidatesWithNetwork1.py --inputdir $input_dir --scratchdir $scratch_dir --test_data training

In [ ]:
%run src/FindCandidatesWithNetwork1.py --inputdir $input_dir --scratchdir $scratch_dir --test_data validation

In [ ]:
%run src/FindCandidatesWithNetwork1.py --inputdir $input_dir --scratchdir $scratch_dir --test_data testing

## Network 2: False-positive reduction

<img src="static_figures/CNN2.png" alt="Architecture of the network" style="width: 600px; margin-top: 20px; margin-bottom: 10px;">

Figure from Lessmann et al., TMI (2018)

Training the second stage network (this will again take quite some time):

In [ ]:
%run src/TrainNetwork2.py --inputdir $input_dir --scratchdir $scratch_dir --visdom $visdom_server

Apply the second stage network to the test data:

In [ ]:
%run src/ReclassifyCandidatesWithNetwork2.py --inputdir $input_dir --scratchdir $scratch_dir --test_data testing

## Postprocessing

In the paper by van Velzen et al., regiongrowing with a 130 HU threshold was used to match standard calcium scoring. 


In [ ]:
%run src/PostprocessRegionGrowing.py --inputdir $input_dir --scratchdir $scratch_dir --test_data testing

## Calcium scores

Calculate Agatston scores (the most commonly used kind of calcium score) and the calcium volume for the different labels:

In [ ]:
%run src/ComputeCalciumScores.py --inputdir $input_dir --scratchdir $scratch_dir --test_data testing --postprocessed False 
# set postprocessed to True for calculating scores of postprocessed data

We also need to calculate the reference scores from the reference annotations:

In [ ]:
%run src/ComputeCalciumScores.py --inputdir $input_dir --scratchdir $scratch_dir --score_annotations

Now we can use the coronary calcium scores to put the subjects into risk categories and compare how well this assignment agrees between manual and reference scores:

In [ ]:
%run src/EvaluateRiskCategoryAssignment.py --inputdir $input_dir --scratchdir $scratch_dir --test_data testing

## Different reconstruction kernels

In the paper by Lessmann et al., different reconstruction kernels (soft tissue / medium-sharp kernels) were separated to investigate whether the method works better on images with soft reconstructions.

All of the external scripts executed above accept an additional parameter "--kernels soft" or "--kernels sharp" to limit the analysis to images reconstructed with either soft or sharp kernels.